### Title
### Dataset

In [4]:
from csv import reader
file = open("hacker_news.csv")
hn = list(reader(file))
print(hn[:5])

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'], ['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']]


In [6]:
# ヘッダー削除
headers = hn[0]
hn = hn[1:]
print(headers)
print(hn[:5])

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
[['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12'], ['10482257', 'Title II kills investment? Comcast and other ISPs are now spending more', 'http:

In [12]:
# ポスト抽出
ask_posts = []
show_posts = []
others_posts = []
for row in hn:
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        others_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(others_posts))
for row in ask_posts[:5]:
    print(row)
for row in show_posts[:5]:
    print(row)

1744
1162
17193
['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']
['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']
['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']
['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']
['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']
['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']
['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']
['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', 

In [14]:
# 平均コメント数よりも多くのコメントをもらっているか？
total_ask_comments = 0
for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments
avg_ask_comments = total_ask_comments / len(ask_posts)
print(avg_ask_comments)
total_show_comments = 0
for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments
avg_show_comments = total_show_comments / len(show_posts)
print(avg_show_comments)

14.038417431192661
10.31669535283993


"Ask HN"のほうが"Show HN"よりも一投稿あたり多くの平均コメントをもらっている．そのため，Ask HNの投稿に絞って分析を行う．

次に，*いつ頃投稿されると，より多くのコメントを貰えるか*分析する

1. 24時間を1時間ごとに区切ってask投稿数をカウントする
2. 1時間ごとのask投稿数における平均コメント数をカウントする


In [25]:
# 1. 24時間を1時間ごとに区切ってask投稿数をカウントする
import datetime as dt
result_list = []
for row in ask_posts:
    created_at = row[6]
    num_comments = int(row[4])
    inner_list = []
    inner_list.append(created_at)
    inner_list.append(num_comments)
    result_list.append(inner_list)
print(result_list[:5])

counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    hour_of_date = dt.datetime.strptime(row[0], "%m/%d/%Y %H:%M")
    hour_of_date = hour_of_date.strftime("%H")
    if hour_of_date not in counts_by_hour:
        counts_by_hour[hour_of_date] = 1
        comments_by_hour[hour_of_date] = row[1]
    elif hour_of_date in counts_by_hour:
        counts_by_hour[hour_of_date] += 1
        comments_by_hour[hour_of_date] += row[1]
print(counts_by_hour)
print(comments_by_hour)

[['8/16/2016 9:55', 6], ['11/22/2015 13:43', 29], ['5/2/2016 10:14', 1], ['8/2/2016 14:20', 3], ['10/15/2015 16:38', 17]]
{'08': 48, '05': 46, '07': 34, '23': 68, '09': 45, '04': 47, '11': 58, '15': 116, '00': 55, '13': 85, '18': 109, '06': 44, '17': 100, '03': 54, '10': 59, '02': 58, '12': 73, '19': 110, '20': 80, '21': 109, '01': 60, '22': 71, '16': 108, '14': 107}
{'08': 492, '05': 464, '07': 267, '23': 543, '09': 251, '04': 337, '11': 641, '15': 4477, '00': 447, '13': 1253, '18': 1439, '06': 397, '17': 1146, '03': 421, '10': 793, '02': 1381, '12': 687, '19': 1188, '20': 1722, '21': 1745, '01': 683, '22': 479, '16': 1814, '14': 1416}


In [44]:
# 2. 1時間ごとのask投稿数における平均コメント数をカウントする
from pprint import pprint
avg_by_hour = []

for hour in counts_by_hour:
    avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])

avg_by_hour.sort()
pprint(avg_by_hour)

[['00', 8.127272727272727],
 ['01', 11.383333333333333],
 ['02', 23.810344827586206],
 ['03', 7.796296296296297],
 ['04', 7.170212765957447],
 ['05', 10.08695652173913],
 ['06', 9.022727272727273],
 ['07', 7.852941176470588],
 ['08', 10.25],
 ['09', 5.5777777777777775],
 ['10', 13.440677966101696],
 ['11', 11.051724137931034],
 ['12', 9.41095890410959],
 ['13', 14.741176470588234],
 ['14', 13.233644859813085],
 ['15', 38.5948275862069],
 ['16', 16.796296296296298],
 ['17', 11.46],
 ['18', 13.20183486238532],
 ['19', 10.8],
 ['20', 21.525],
 ['21', 16.009174311926607],
 ['22', 6.746478873239437],
 ['23', 7.985294117647059]]


In [60]:
swap_avg_by_hour = []
for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
# pprint(swap_avg_by_hour)
sorted_swap = sorted(swap_avg_by_hour, reverse=True)
# pprint(sorted_swap)
print("Top 5 Hours for Ask Posts Comments")
for row in sorted_swap[:5]:
    avg = row[0]
    hour = dt.datetime.strptime(row[1],"%H")
    hour = hour.strftime("%H:%M")
    str = "{1}: {0:.2f} average comments per post".format(avg, hour)
    print(str)


Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post
02:00: 23.81 average comments per post
20:00: 21.52 average comments per post
16:00: 16.80 average comments per post
21:00: 16.01 average comments per post
